In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "/content/drive/MyDrive/CS544-NLP"

/content/drive/MyDrive/CS544-NLP


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git@main accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.4 MB/s eta 0:00:00


### Loading the model and tokenizer

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch
from tqdm import notebook
import pandas as pd
import numpy as np

In [ ]:
model_id = "codellama/CodeLlama-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)

### Preparing the Pipeline

In [ ]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Generating Code

In [ ]:
def generate_code(prompt):
  print(prompt)
  sequences = pipeline(
      prompt,
      do_sample=True,
      temperature=0.1,
      top_p=0.9,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      max_length=512
  )
  response = ""
  for seq in sequences:
      print(f"Result: {seq['generated_text']}")
      response += seq['generated_text']
  return response

In [ ]:
def extract_code(response):
  # lines = response.split('\n')
  # code = '\n'.join(lines)
  # return code
  return response

def save_file(path, code):
  with open(path, 'w') as file:
      file.write(code)

def run_code_generation(df, save_path, n_passes=5):
  for index, row in notebook.tqdm(df.iterrows(), total=df.shape[0]):
    question = row['question']
    code_context = row['code_context']
    for p in range(n_passes):
      prompt = f"# {question} \n{code_context}\n"
      response = generate_code(prompt)
      code = extract_code(response)
      save_file(f"{save_path}/question-{index}-pass-{p}.py", code)

In [ ]:
CODE_GENERATION_PATH = "./generations/python"

In [ ]:
train_df = pd.read_csv('./data/python/train_processed.csv', index_col=0)
train_df.head()

,text,code,question,functions,code_context
id,,,,,
0,Maximum Prefix Sum possible by merging two giv...,"def maxPresum(a, b):\n X = max(a[0], 0)\n ...",Maximum Prefix Sum possible by merging two giv...,"['def maxPresum(a, b):']","def maxPresum(a, b):"
1,Check if a number can be represented as sum of...,import math\n\n\ndef sumOfTwoCubes(n):\n lo...,Check if a number can be represented as sum of...,['def sumOfTwoCubes(n):'],import math\n\n\ndef sumOfTwoCubes(n):
2,Generate an N | Python3 program for the above ...,sieve = [1] * (1000000 + 1)\n\n\ndef sieveOfPr...,Generate an N.,"['def sieveOfPrimes():', 'def getArray(arr, N):']",sieve = [1] * (1000000 + 1)\n\n\ndef sieveOfPr...
3,Nth natural number after removing all numbers ...,def findNthNumber(N):\n result = 0\n p =...,Nth natural number after removing all numbers ...,['def findNthNumber(N):'],def findNthNumber(N):
4,Check if an integer is rotation of another giv...,"import math\n\n\ndef check(A, B):\n if (A =...",Check if an integer is rotation of another giv...,"['def check(A, B):']","import math\n\n\ndef check(A, B):"


In [ ]:
def build_question_context(text, context):
    q_strs = text.split('|')
    q_strs = q_strs[1].split(';')[1:]
    # comments = []
    # for i in range(1, len(q_strs)):
    #     comments.append(f'\\ {i}. {q_strs[i]}')

    # return ' '.join(comments) + '\n' + context
    return context

In [ ]:
train_df_100 = train_df[199:200].copy()
train_df_100['context'] = train_df_100.apply(lambda row: build_question_context(row.text, row.code_context), axis=1)
train_df_100.head()

,text,code,question,functions,code_context,context
id,,,,,,
199,Maximum value of | arr [ 0 ] | Function to ret...,def maxValue(n):\n if (n == 1):\n re...,Maximum value of.,['def maxValue(n):'],def maxValue(n):,def maxValue(n):


In [ ]:
run_code_generation(train_df_100, CODE_GENERATION_PATH, n_passes=1)

  0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


# Maximum value of. 
def maxValue(n):

Result: # Maximum value of. 
def maxValue(n):
    if n == 0:
        return 0
    else:
        return n + maxValue(n-1)

# Maximum value of 100
print(maxValue(100))

# Maximum value of 1000
print(maxValue(1000))

# Maximum value of 10000
print(maxValue(10000))

# Maximum value of 100000
print(maxValue(100000))

# Maximum value of 1000000
print(maxValue(1000000))

# Maximum value of 10000000
print(maxValue(10000000))

# Maximum value of 100000000
print(maxValue(100000000))

# Maximum value of 1000000000
print(maxValue(1000000000))

# Maximum value of 10000000000
print(maxValue(10000000000))

# Maximum value of 100000000000
print(maxValue(100000000000))

# Maximum value of 1000000000000
print(maxValue(1000000000000))

# Maximum value of 10000000000000
print(maxValue(10000000000000))

# Maximum value of 100000000000000
print(maxValue(100000000000000))

# Maximum value of 1000000000000000
print(maxValue(1000000000


### Testing on Several Queries

### Code Infilling

For Future...

In [ ]:
from transformers import pipeline
import torch

generator = pipeline("text-generation",model="codellama/CodeLlama-7b-hf",torch_dtype=torch.float16, device_map="auto")
# generator('def remove_non_ascii(s: str) -> str:\n    """ <FILL_ME>\n    return result', max_new_tokens = 128, return_type = 1)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model_id = "codellama/CodeLlama-7b-hf"
tokenizer2 = AutoTokenizer.from_pretrained(model_id)
model2 = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16
).to("cuda")




In [ ]:
prompt = '''def remove_non_ascii(s: str) -> str:
    """ <FILL_ME>
    return result
'''

input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to("cuda")
output = model.generate(
    input_ids,
    max_new_tokens=200,
)
output = output[0].to("cpu")

filling = tokenizer.decode(output[input_ids.shape[1]:], skip_special_tokens=True)


In [ ]:
print(prompt.replace("<FILL_ME>", filling))